# Baidu News Digger
## import librarys

In [1]:
import requests # url requests lib
import re # regular expression 
import pymysql # mysql lib
import time
from datetime import timedelta, datetime

## user-agent define

In [2]:
headers = {'user-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36 Edg/92.0.902.73'}

## Baidu News Spider Define

In [3]:
def baidu(company):
    url = 'https://www.baidu.com/s? tn=news&rtt=4&bsst=1&cl=2&wd=' + company
    res = requests.get(url, headers=headers, timeout=10).text
    try:
        res = res.encode('ISO-8859-1').decode('utf-8')
    except:
        try:
            res = res.encode('ISO-8859-1').decode('gbk')
        except:
            res = res# new function and parameter is company:list
    
    """Regular Expression"""
    p_href = '<h3 class="news-title_1YtI1"><a href="(.*?)"'
    p_title = '<h3 class="news-title_1YtI1">.*?>(.*?)</a>'
    p_info = '<div class="news-source_Xj4Dv">(.*?)</div>'
    href = re.findall(p_href, res, re.S)
    title = re.findall(p_title, res, re.S)
    info = re.findall(p_info, res, re.S)
    
    """Split source and date from info"""
    source = []
    date = []
    for i in range(len(title)):
        title[i] = title[i].strip()
        title[i] = re.sub('<.*?>', '', title[i])
        info[i] = info[i].strip()
        info[i] = re.sub('<.*?>', '', info[i])
        source.append(info[i].split('\n')[0])
        date.append(info[i].split('\n')[1])
        source[i] = source[i].strip()
        date[i] = date[i].strip()
    
        """Time Formatting"""
        if ('小时' in date[i] or ('分钟') in date[i]):
            date[i] = time.strftime("%Y-%m-%d")
        elif (':' in date[i] or ('今天') in date[i] or ('昨天') in date[i]):
            yesterday = datetime.today() + timedelta(-1)
            date[i] = yesterday.strftime("%Y-%m-%d")
        
    """Score the news and date deep clean"""   
    score = []
    keywords = ['违约','诉讼','兑付','大跌','蒸发', '逃离']
    
    for i in range(len(title)):
        num = 0
        try:
            article = requests.get(href[i], headers=headers, timeout=10).text
        except:
            print(commpany +" Article Digging failure")
        
        try:
            article = article.encode('ISO-8859-1').decode('uft-8')
        except:
            try:
                article = article.encode('ISO-8859-1').decode('gbk')
            except:
                article = article
                
        p_article = '<p>(/*?)</p>'
        article_main = re.findall(p_article, article)
        article = ''.join(article_main)
        
        for k in keywords:
            if (k in article) or (k in title[i]):
                num -= 5
        score.append(num)
        
#         """Deep Clean"""
#         company_re = company[0] + '.{0, 5}' + company[-1]
#         if len(re.findall(company_re, article)) < 1:
#             title[i] = ''
#             source[i] = ''
#             href[i] = ''
#             date[i] = ''
#             score[i] = ''
#     while '' in title:
#         title.remove('')
#     while '' in href:
#         href.remove('')
#     while '' in date:
#         date.remove('')
#     while '' in source:
#         source.remove('')
#     while '' in score:
#         score.remove('')
    
    """Print date after cleaned"""
    for i in range(len(title)):
        print(str(i+1) + '.' + title[i] + '(' + date[i] + ' ' + source[i] + ')')
        print(href[i])
        print(company + '(' + '舆情分数: ' + str(score[i]) + ')')
        
    '''Save data to Mysql'''
    for i in range(len(title)):
        db = pymysql.connect(host='localhost', user='root', database='spidder')
        cur = db.cursor()
#         """Query data and distinct"""
#         sql_1 = 'SELECT * FROM baidunews WHERE company = %s'
#         cur.execute(sql_1, company)
#         data_all = cur.fetchall()
#         title_all = []
#         for j in range(len(data_all)):
#             title_all.append(data_all[j][1])
            
#         if title[i] not in title_all:
        sql_2 = 'INSERT INTO baidunews(company, title, href, source, date, score) VALUES (%s, %s, %s, %s, %s,%s)'
        cur.execute(sql_2, (company, title[i], href[i], source[i], date[i], score[i]))
        db.commit()
        cur.close()
        db.close()
        
    print('END--------------------------------------------------------')

In [4]:
companys = ['水井坊', '阿里巴巴', '腾讯集团']
for company in companys:
    try:
        baidu(company)
        print(company + ' Data digging succeesul')
    except:
        print(company + ' Data digging failure')

1.水井坊8月26日盘中跌幅达5%(2021-08-26 东方财富网)
http://stock.eastmoney.com/a/202108262066260068.html
水井坊(舆情分数: 0)
2.水井坊8月26日快速反弹(2021-08-26 东方财富网)
http://stock.eastmoney.com/a/202108262065464660.html
水井坊(舆情分数: 0)
3.水井坊:8月25日融资净买入2383.83万元 环比增加680.42%(2021-08-26 同花顺财经)
http://stock.10jqka.com.cn/20210826/c632168329.shtml
水井坊(舆情分数: 0)
4.高端白酒塑造将添“水井坊模式”,以专注与天赋构建竞争新格局(2021-08-25 腾讯网)
https://new.qq.com/omn/20210825/20210825A0ABTU00.html
水井坊(舆情分数: 0)
5.大商组建高端白酒销售公司 水井坊“破局”高端化|水井坊_新浪财经...(2021-08-25 新浪财经)
http://finance.sina.com.cn/stock/relnews/cn/2021-08-25/doc-ikqcfncc4919505.shtml
水井坊(舆情分数: 0)
6.迈入专营时代,水井坊引领高端白酒渠道新变局(2021-08-25 新浪财经)
http://finance.sina.com.cn/jjxw/2021-08-25/doc-ikqciyzm3520258.shtml
水井坊(舆情分数: 0)
7.迈入专营时代 水井坊引领高端白酒渠道新变局|白酒_新浪财经_新浪网(2021-08-25 新浪财经)
http://finance.sina.com.cn/stock/relnews/cn/2021-08-25/doc-ikqciyzm3526333.shtml
水井坊(舆情分数: 0)
8.白酒板块“发烧”之后,向左还是向右?|水井坊|贵州茅台|金徽酒...(2021-08-25 新浪财经)
http://finance.sina.com.cn/tech/2021-08-25/doc-ikqcfncc4899154.shtml
水井坊(舆情分数: 0)
9.大商组建高